In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_07 import *

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
mnist_view = view_tfm(1, 28, 28)
cbfs = [Recorder,
       partial(AvgStatsCallback, accuracy),
       CudaCallback,
       partial(BatchTransformXCallback, mnist_view)]

In [5]:
nfs = [8, 16, 32, 64, 64]

In [8]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, stride, padding=ks//2, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
        
    def forward(self, x): return self.relu(self.conv(x))

    @property
    def bias(self): return -self.relu.sub
    @bias.setter
    def bias(self, v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [9]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbfs)

In [10]:
run.fit(2, learn)

train: [1.511159375, tensor(0.4879, device='cuda:0')]
valid: [0.439822900390625, tensor(0.8708, device='cuda:0')]
train: [0.17279075520833334, tensor(0.9460, device='cuda:0')]
valid: [0.12580836181640626, tensor(0.9623, device='cuda:0')]


In [11]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbfs)

In [15]:
#export 
def get_batch(dl, run):
    run.xb, run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb, run.yb

In [16]:
xb, yb = get_batch(data.train_dl, run)

In [25]:
#export
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o, cond) for o in m.children()], [])

In [26]:
#export
def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [27]:
mods = find_modules(learn.model, lambda o: isinstance(o, ConvLayer))

In [28]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

In [53]:
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean, hook.std = d.mean().item(), d.std().item()

In [54]:
mdl = learn.model.cuda()

In [55]:
with Hooks(mdl, append_stat) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean, hook.std)

0.39001157879829407 0.7755716443061829
0.4937850832939148 0.8303635716438293
0.37032121419906616 0.6824780702590942
0.3763061463832855 0.5989781618118286
0.24864579737186432 0.3572244644165039
0.24864579737186432 0.3572244644165039
0.24864579737186432 0.3572244644165039
0.07103578001260757 0.3224029839038849


In [56]:
xb.mean(), xb.std()

(tensor(0.0034, device='cuda:0'), tensor(1.0041, device='cuda:0'))

In [58]:
#export
def lsuv_module(m, xb):
    h = Hook(m, append_stat)
    
    while mdl(xb) is not None and abs(h.mean) > 1e-3: m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std - 1) > 1e-3: m.weight.data /=h.std
        
    h.remove()
    return h.mean, h.std

In [59]:
for m in mods: print(lsuv_module(m, xb))

(0.11285822838544846, 1.0)
(0.01909920759499073, 0.9999998807907104)
(0.15176896750926971, 0.9999999403953552)
(0.1386193335056305, 0.9999999403953552)
(0.28584790229797363, 1.0)


In [60]:
%time run.fit(2, learn)

train: [0.4309011393229167, tensor(0.8622, device='cuda:0')]
valid: [0.1174605224609375, tensor(0.9627, device='cuda:0')]
train: [0.0897241455078125, tensor(0.9716, device='cuda:0')]
valid: [0.11510345458984375, tensor(0.9636, device='cuda:0')]
CPU times: user 2.79 s, sys: 382 ms, total: 3.17 s
Wall time: 2.75 s


In [61]:
!python notebook2script.py 07a_LSUV.ipynb

Converted 07a_LSUV.ipynb to exp/nb_07a.py
